Aquí se importan las librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
import time

Función Monte Carlo Iterativa
-Se genera una cantidad de num_puntos puntos aleatorios dentro de un rectángulo que cubre la función en el intervalo [a, b].
-Se cuentan los puntos que están debajo de la curva de la función.
-La integral se estima como el área del rectángulo multiplicada por la proporción de puntos que cayeron debajo de la función.

In [ ]:
# Función Monte Carlo iterativa
def integra_mc_iterativa(fun, a, b, num_puntos=10000):
    """
    Calcula la integral de una función usando el método de Monte Carlo (iterativo).
    Parametros:
        fun: función que vamos a integrar.
        a, b: límites de integración.
        num_puntos: número de puntos aleatorios.
        
    Returns:
        Aproximación de la integral.
    """
    M = max(fun(np.linspace(a, b, 1000)))  # Valor máximo de la función (aproximado)
    debajo = 0  # Contador de puntos bajo la curva
    puntos_x = []  # Almacena las coordenadas x de los puntos generados
    puntos_y = []  # Almacena las coordenadas y de los puntos generados
    for _ in range(num_puntos):
        x = np.random.uniform(a, b)  # Generar x aleatorio
        y = np.random.uniform(0, M)  # Generar y aleatorio
        puntos_x.append(x)
        puntos_y.append(y)
        if y < fun(x):  # Verificar si el punto está bajo la curva
            debajo += 1 #Aumenta los que estan debajo
    return (debajo / num_puntos) * (b - a) * M, puntos_x, puntos_y

Función de Monte Carlo Vectorizada
-En lugar de generar puntos uno por uno, usamos NumPy para generar todos los puntos de una sola vez.
-Esto elimina la necesidad de bucles y mejora enormemente la velocidad del cálculo.

In [ ]:
# Función Monte Carlo vectorizada
def integra_mc_vectorizada(fun, a, b, num_puntos=10000):
    """
    Calcula la integral de una función usando el método de Monte Carlo (vectorizado).
    
    Parametros:
        fun: función a integrar.
        a, b: límites de integración.
        num_puntos: número de puntos aleatorios.
        
    Returns:
        Aproximación de la integral y las coordenadas de los puntos generados.
    """
    M = max(fun(np.linspace(a, b, 1000)))  # Valor máximo de la función (aproximado)
    x = np.random.uniform(a, b, num_puntos)  # Generar x aleatorios
    y = np.random.uniform(0, M, num_puntos)  # Generar y aleatorios
    debajo = np.sum(y < fun(x))  # Contar puntos bajo la curva
    return (debajo / num_puntos) * (b - a) * M, x, y  # Devolver la integral y los puntos

-Se ejecutan ambos métodos y se mide cuánto tiempo tardan.
-Se compara su resultado con el método quad de SciPy, que es muy preciso.
-Se calcula el error relativo de cada método con respecto al valor exacto.
-Se genera una gráfica para comparar visualmente los tiempos de ejecución.
-Se visualiza la función junto con los puntos generados para entender cómo funciona el método Monte Carlo.

In [ ]:
# Comparación de tiempos y precisión
def comparar_metodos(fun, a, b, num_puntos=10000):
    """
    Compara los métodos iterativo y vectorizado de Monte Carlo con scipy.integrate.quad.
    """
    # Tiempo y resultado del método iterativo
    start = time.time()
    resultado_iter, puntos_x_iter, puntos_y_iter = integra_mc_iterativa(fun, a, b, num_puntos)
    tiempo_iter = time.time() - start

    # Tiempo y resultado del método vectorizado
    start = time.time()
    resultado_vec, puntos_x_vec, puntos_y_vec = integra_mc_vectorizada(fun, a, b, num_puntos)
    tiempo_vec = time.time() - start

    # Resultado exacto con scipy.integrate.quad
    resultado_exacto, _ = quad(fun, a, b)

    # Imprimir resultados
    print(f"Resultado iterativo: {resultado_iter:.6f}, Tiempo: {tiempo_iter:.6f} s")
    print(f"Resultado vectorizado: {resultado_vec:.6f}, Tiempo: {tiempo_vec:.6f} s")
    print(f"Resultado exacto (quad): {resultado_exacto:.6f}")

    # Error relativo
    error_iter = abs(resultado_iter - resultado_exacto) / abs(resultado_exacto)
    error_vec = abs(resultado_vec - resultado_exacto) / abs(resultado_exacto)
    print(f"Error iterativo: {error_iter:.6%}")
    print(f"Error vectorizado: {error_vec:.6%}")

    # Gráfica de comparación de tiempos
    labels = ['Iterativo', 'Vectorizado']
    tiempos = [tiempo_iter, tiempo_vec]
    plt.bar(labels, tiempos, color=['red', 'blue'])
    plt.ylabel('Tiempo (s)')
    plt.title('Comparación de tiempos')
    plt.show()

    # Graficar la función, el área bajo la curva y los puntos
    graficar_funcion_y_puntos(fun, a, b, puntos_x_iter, puntos_y_iter)

# Relación entre número de puntos y error relativo
def analizar_errores(fun, a, b, max_puntos=100000):
    """
    Analiza cómo varía el error relativo con el número de puntos.
    """
    puntos = np.logspace(2, np.log10(max_puntos), 10, dtype=int)
    errores_iter = []
    errores_vec = []
    tiempos_iter = []
    tiempos_vec = []
    resultado_exacto, _ = quad(fun, a, b)

    for num_puntos in puntos:
        # Iterativo
        start = time.time()
        resultado_iter,_,_ = integra_mc_iterativa(fun, a, b, num_puntos)
        tiempos_iter.append(time.time() - start)
        errores_iter.append(abs(resultado_iter - resultado_exacto) / abs(resultado_exacto))

        # Vectorizado
        start = time.time()
        resultado_vec,_,_ = integra_mc_vectorizada(fun, a, b, num_puntos)
        tiempos_vec.append(time.time() - start)
        errores_vec.append(abs(resultado_vec - resultado_exacto) / abs(resultado_exacto))

    # Gráfica de errores relativos
    plt.figure(figsize=(12, 6))
    plt.plot(puntos, errores_iter, label='Iterativo', marker='o', color='red')
    plt.plot(puntos, errores_vec, label='Vectorizado', marker='o', color='blue')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Número de puntos')
    plt.ylabel('Error relativo')
    plt.title('Error relativo vs Número de puntos')
    plt.legend()
    plt.show()

    # Gráfica de tiempos
    plt.figure(figsize=(12, 6))
    plt.plot(puntos, tiempos_iter, label='Iterativo', marker='o', color='red')
    plt.plot(puntos, tiempos_vec, label='Vectorizado', marker='o', color='blue')
    plt.xscale('log')
    plt.xlabel('Número de puntos')
    plt.ylabel('Tiempo (s)')
    plt.title('Tiempo de ejecución vs Número de puntos')
    plt.legend()
    plt.show()

# Graficar la función, el área bajo la curva y los puntos
def graficar_funcion_y_puntos(fun, a, b, puntos_x, puntos_y):
    """
    Muestra una gráfica de la función, el área acotada y los puntos generados.
    """
    # Crear un conjunto de puntos para la función f(x)
    x_vals = np.linspace(a, b, 1000)
    y_vals = fun(x_vals)

    # Crear la figura y los ejes
    plt.figure(figsize=(10, 6))

    # Graficar la función f(x)
    plt.plot(x_vals, y_vals, label="f(x)", color="green", linewidth=2)

    # Graficar los puntos aleatorios
    puntos_bajo = [puntos_y[i] for i in range(len(puntos_y)) if puntos_y[i] < fun(puntos_x[i])]
    puntos_arriba = [puntos_y[i] for i in range(len(puntos_y)) if puntos_y[i] >= fun(puntos_x[i])]
    plt.scatter([puntos_x[i] for i in range(len(puntos_x)) if puntos_y[i] < fun(puntos_x[i])], 
                puntos_bajo, color='red', label="Puntos bajo la curva", s=10)
    plt.scatter([puntos_x[i] for i in range(len(puntos_x)) if puntos_y[i] >= fun(puntos_x[i])], 
                puntos_arriba, color='blue', label="Puntos fuera de la curva", s=10)

    # Resaltar el área bajo la curva
    plt.fill_between(x_vals, 0, y_vals, color='yellow', alpha=0.3, label="Área bajo la curva")

    # Etiquetas y leyenda
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.title(f'Gráfica de la función f(x) y puntos generados (Método de Monte Carlo)')
    plt.legend()

    plt.show()

-Definimos una función de prueba (sin(x) en valor absoluto).
-Definimos el intervalo de integración [0,5].
-Se ejecuta la función comparar_metodos() para evaluar los diferentes métodos de integración.

In [1]:
if __name__ == "__main__":
    # Definir una función a integrar, en este caso f(x) = x^2
    def f(x):
        return np.abs(np.sin(x))

    # Límites de integración
    a, b = 0, 5

    # Número de puntos
    num_puntos = 10000

    # Comparar métodos
    print("Comparación de métodos:")
    comparar_metodos(f, a, b, num_puntos)

    # Analizar errores
    print("\nAnálisis de errores:")
    analizar_errores(f, a, b, max_puntos=100000)

Comparación de métodos:


NameError: name 'comparar_metodos' is not defined